Analysis and Verification of Multi-hop Cumulative Time Synchronization Error in TSN Networks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# 参数设置
max_hops = 7
num_samples = 1000  # 每跳的采样次数
timestamp_error = 12e-9  # 时间戳误差，单位：秒
max_freq_ratio_error = 0.1e-6  # 相邻节点频率比最大误差（0.1ppm）
max_residence_time = 1e-3  # 最大驻留时间，单位：秒

def simulate_synchronization_error(hop_count):
    """
    模拟特定跳数的时间同步误差
    
    参数:
    hop_count -- 跳数
    
    返回:
    cumulative_errors -- 累积同步误差数组（单位：纳秒）
    """
    # 初始化数组存储每个样本的累积误差
    cumulative_errors = np.zeros(num_samples)
    
    for sample in range(num_samples):
        # 初始化当前样本的累积误差
        C_i_prev = 0  # 前一节点的修正域误差
        R_mi_cum_error = 0  # 主时钟到当前节点的累积频率比误差
        
        for i in range(1, hop_count + 1):
            # 生成此跳的频率比误差（假设在±max_freq_ratio_error范围内均匀分布）
            r_error = np.random.uniform(-max_freq_ratio_error, max_freq_ratio_error)
            
            # 链路传播延迟误差（基于公式11）
            # 𝛿𝐷ᵢ = 𝛿𝑡 + (1+𝛿𝑅ᵢ₋₁,ᵢ)×(𝛿𝑡₂-𝛿𝑡₃) + 2×𝛿𝑡
            t_error = np.random.uniform(-timestamp_error, timestamp_error)
            t2_error = np.random.uniform(-timestamp_error, timestamp_error)
            t3_error = np.random.uniform(-timestamp_error, timestamp_error)
            D_i_error = t_error + (1 + r_error) * (t2_error - t3_error) + 2 * t_error
            
            # 驻留时间误差（假设为2倍时间戳误差）
            S_i_error = 2 * t_error
            
            # 主时钟到当前节点的累积频率比误差（基于公式10）
            # 𝛿𝑅ₘ,ᵢ = ∑ⱼ₌₁ⁱ(𝑟ⱼ×∏ₖ₌ⱼ₊₁ⁱ𝑟ₖ)
            if i == 1:
                R_mi_cum_error = r_error
            else:
                R_mi_cum_error = R_mi_cum_error + r_error * (1 + R_mi_cum_error)
            
            # 修正域误差（基于公式12）
            # 𝛿𝐶ᵢ = 𝛿𝐶ᵢ₋₁ + 𝛿𝐷ᵢ + 𝛿𝑆ᵢ×𝑅ₘ,ᵢ₋₁ + 𝛿𝑅ₘ,ᵢ₋₁×(𝑖-1)×𝑡
            C_i = C_i_prev + D_i_error + S_i_error * (1 + R_mi_cum_error) + R_mi_cum_error * (i-1) * timestamp_error
            C_i_prev = C_i
            
            # 更新当前跳数的累积误差（基于公式8）
            # 𝛿GM,ᵢ(t) = 𝛿𝐶ᵢ₋₁ + 𝛿𝐷ᵢ×𝑟ᵢ₋₁,ᵢ
            if i == hop_count:
                cumulative_errors[sample] = (C_i_prev + D_i_error * (1 + r_error)) * 1e9  # 转换为纳秒
    
    return cumulative_errors

def calculate_theoretical_error(hop_count):
    """
    计算理论最大同步误差（单位：纳秒）
    """
    # 基于论文中的理论计算值
    theoretical_data = {
        1: 72,
        2: 120,
        3: 168,
        4: 217,
        5: 266,
        6: 315,
        7: 364
    }
    
    return theoretical_data.get(hop_count, 0)

# 运行模拟并收集结果
max_errors = []
min_errors = []
mean_errors = []
theoretical_errors = []

for hops in range(1, max_hops + 1):
    print(f"模拟 {hops} 跳的同步误差...")
    errors = simulate_synchronization_error(hops)
    
    max_errors.append(np.max(errors))
    min_errors.append(np.min(errors))
    mean_errors.append(np.mean(errors))
    theoretical_errors.append(calculate_theoretical_error(hops))
    
    print(f"  最大误差: {max_errors[-1]:.2f} ns")
    print(f"  最小误差: {min_errors[-1]:.2f} ns")
    print(f"  平均误差: {mean_errors[-1]:.2f} ns")
    print(f"  理论最大误差: {theoretical_errors[-1]} ns")

# 绘制结果
plt.figure(figsize=(10, 6))
hops_range = list(range(1, max_hops + 1))

plt.plot(hops_range, max_errors, 'ro-', label='最大测试误差')
plt.plot(hops_range, mean_errors, 'go-', label='平均测试误差')
plt.plot(hops_range, min_errors, 'bo-', label='最小测试误差')
plt.plot(hops_range, theoretical_errors, 'k--', label='理论最大误差')

plt.xlabel('跳数')
plt.ylabel('同步误差 (ns)')
plt.title('TSN网络多跳累积时间同步误差')
plt.grid(True)
plt.legend()
plt.xticks(hops_range)

# 输出最后一跳的误差分布图
plt.figure(figsize=(10, 6))
last_hop_errors = simulate_synchronization_error(max_hops)
plt.hist(last_hop_errors, bins=30, alpha=0.7, color='blue')
plt.axvline(np.mean(last_hop_errors), color='r', linestyle='dashed', linewidth=2, label=f'平均值: {np.mean(last_hop_errors):.2f} ns')
plt.axvline(theoretical_errors[-1], color='k', linestyle='dashed', linewidth=2, label=f'理论最大值: {theoretical_errors[-1]} ns')
plt.xlabel('同步误差 (ns)')
plt.ylabel('频率')
plt.title(f'{max_hops} 跳累积误差分布')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 模拟NRR和CF的稳定性随跳数增加而降低
def simulate_nrr_cf_stability():
    """模拟NRR和CF的稳定性随跳数增加而变化"""
    hops_to_show = [2, 7]  # 对应于1个交换机和6个交换机的情况
    time_points = 100  # 时间点数量
    
    plt.figure(figsize=(12, 10))
    
    # NRR稳定性
    for hop_idx, hop in enumerate(hops_to_show):
        # NRR稳定性随跳数增加而下降，振幅增大
        base_nrr_error = 0.01e-6 * hop  # 基准误差随跳数增加
        nrr_noise = np.random.normal(0, base_nrr_error, time_points) * 1e6  # 转为ppm
        
        plt.subplot(2, 2, hop_idx+1)
        plt.plot(range(time_points), nrr_noise, 'b-')
        plt.axhline(y=0.1, color='r', linestyle='--', label='阈值 (0.1ppm)')
        plt.axhline(y=-0.1, color='r', linestyle='--')
        plt.xlabel('时间')
        plt.ylabel('NRR 精度 (ppm)')
        plt.title(f'NRR 精度 - {hop-1} 个交换机')
        plt.grid(True)
        plt.legend()
    
    # CF稳定性
    for hop_idx, hop in enumerate(hops_to_show):
        # CF稳定性随跳数增加而下降，振幅增大
        base_cf_error = 20 * (hop/2)  # 基准误差随跳数增加
        cf_noise = np.random.normal(0, base_cf_error, time_points)
        
        plt.subplot(2, 2, hop_idx+3)
        plt.plot(range(time_points), cf_noise, 'g-')
        plt.xlabel('时间')
        plt.ylabel('CF 精度 (ns)')
        plt.title(f'CF 精度 - {hop-1} 个交换机')
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# 模拟NRR和CF稳定性
simulate_nrr_cf_stability()